# Install Libraries

In [1]:
# py -m pip install -r requirements.txt

# Import Libraries

In [2]:
import json
from openai import OpenAI
import os
import pandas as pd
import utils
import numpy as np

# Load Data

In [3]:
clauses=pd.read_csv("C:\\Users\marth\\Data Science\\Ciba\\clauses - clauses.csv")

# Test Extraction Function

In [ ]:
clause_in=clauses['Clause'].values[0]
tst_output=json.loads(utils.extract_insurance_info(clause_in))
tst_output

# Get Relevant columns

In [4]:
clauses_to_compare=clauses[[
 'GeneralLiability_Required',
 'GeneralLiability_EachOccurrence',
 'GeneralLiability_GeneralAggregate',
#  conditional_coverage
#  'GeneralLiability_AddlInsd',
# 'AI_CommercialGeneralLiability',

 'AutoLiability_Required',
#   'Missing Occurrence',
#  'Missing GeneralAggregate',
 'AutoLiability_Conditional',
#  'AI_AutoLiability',
#  'AI_AutoLiability_Entity1',


 'WorkersCompensation_Required',
#   'Missing _EachOccurrence',
#  'Missing GeneralAggregate',
 'WorkersCompensation_Conditional',
#  'AI_WorkersCompensation',
#  'AI_WorkersCompensation_Entity1',
 
 'ProfessionalLiability_Required',
 'ProfessionalLiability_EachOccurrence',
 'ProfessionalLiability_GeneralAggregate',
#  'Missing _Conditional',
#   'AI_Missing ',
#  'Missing _Entity1',
 ]]
clauses_to_compare.head(2)
# clauses.columns.to_list()

,GeneralLiability_Required,GeneralLiability_EachOccurrence,GeneralLiability_GeneralAggregate,AutoLiability_Required,AutoLiability_Conditional,WorkersCompensation_Required,WorkersCompensation_Conditional,ProfessionalLiability_Required,ProfessionalLiability_EachOccurrence,ProfessionalLiability_GeneralAggregate
0,Yes,"2,000,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Yes,"$1,000,000","2,000,000",Yes,NaN,Yes,NaN,NaN,NaN,NaN


# Clean currency

In [14]:
def clean_currency_columns(df, columns):
    """
    Remove dollar signs and commas from specified columns in a pandas DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the columns to clean.
    columns (list): A list of column names to clean.

    Returns:
    pd.DataFrame: A DataFrame with the specified columns cleaned.
    """
    df = df.copy()  # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    for column in columns:
        if column in df.columns:
            # Remove dollar signs and commas, convert to numeric
            # df[column] = pd.to_numeric(df[column].astype(str).str.replace('[\$,]', '', regex=True), errors='coerce')
            df[column] = df[column].astype(str).str.replace('[\$,]', '', regex=True)
        else:
            print(f"Column '{column}' does not exist in the DataFrame.")
    return df

clean_clause=clean_currency_columns(clauses_to_compare, [

 'GeneralLiability_EachOccurrence',
 'GeneralLiability_GeneralAggregate',
 'ProfessionalLiability_EachOccurrence',
 'ProfessionalLiability_GeneralAggregate',

 ])



# To Validate Results

## Steps

1. **Convert JSON Output to DataFrame**
    - Transform the JSON output received from GPT-4o into a DataFrame for easier manipulation and analysis.

2. **Get Relevant Columns from Original Dataset**
    - Extract the necessary columns from the original dataset that correspond to the information extracted by GPT-4o.

3. **Align Column Names**
    - Ensure that the columns in both the extracted DataFrame and the original dataset have identical names. This facilitates direct comparison and validation.

4. **Standardize Columns**
    - Standardize the data in the columns for consistency. For example, some dollar amounts may include a '$' symbol while others do not. 

# Full Run

In [8]:
clauses_extracted=clauses
clause_in=clauses['Clause'].values
clauses_extracted['extracted_info']=[utils.extract_insurance_info(x) for x in clause_in]


## Convert Json to pandas df and Standardize Columns

In [31]:
data_dicts = [json.loads(json_str) for json_str in clauses_extracted['extracted_info'].values]
normalized_data=[pd.json_normalize(data) for data in data_dicts]
df = pd.concat(normalized_data, ignore_index=True)[[
 'insurance_requirements.general_liability.coverage_required',
 'insurance_requirements.general_liability.each_occurrence_limit',
 'insurance_requirements.general_liability.aggregate_limit',
 'insurance_requirements.automobile_liability.coverage_required',
 'insurance_requirements.automobile_liability.conditional_coverage',
 'insurance_requirements.workers_compensation.coverage_required',
 'insurance_requirements.workers_compensation.conditional_coverage',
 'insurance_requirements.professional_liability.coverage_required',
 'insurance_requirements.professional_liability.each_occurrence_limit',
 'insurance_requirements.professional_liability.aggregate_limit']]

import numpy as np
for column in df.columns:
    print()
    if df[column].dtype == 'object':  # Column is a string
        df[column] = df[column].replace(['No', 'None'], np.nan)
    elif pd.api.types.is_numeric_dtype(df[column]):  # Column is numeric
        df[column] = pd.to_numeric(df[column], errors='coerce').replace(-999, np.nan)
    else:  # Convert column to numeric and replace 'No' with NaN 
        df[column] = df[column].replace(['No', 'null'], ['-999', 'nan'])
        df[column] = pd.to_numeric(df[column], errors='coerce').replace(-999, np.nan)

df.columns=clauses_to_compare.columns
clean_extracted=clean_currency_columns(df, [

 'GeneralLiability_EachOccurrence',
 'GeneralLiability_GeneralAggregate',
 'ProfessionalLiability_EachOccurrence',
 'ProfessionalLiability_GeneralAggregate',

 ])
for column in clean_clause.columns:
    print()
    if clean_clause[column].dtype == 'object':  # Column is a string
        clean_clause[column] = clean_clause[column].replace(['No', 'None','nan'], np.nan)
    elif pd.api.types.is_numeric_dtype(clean_clause[column]):  # Column is numeric
        clean_clause[column] = pd.to_numeric(clean_clause[column], errors='coerce').replace(-999, np.nan)
    else:  # Convert column to numeric and replace 'No' with NaN 
        clean_clause[column] = clean_clause[column].replace(['No', 'null'], ['-999', 'nan'])
        clean_clause[column] = pd.to_numeric(clean_clause[column], errors='coerce').replace(-999, np.nan)


comparison = clean_extracted.compare(clean_clause)
(comparison)



GeneralLiability_Required       GeneralLiability_EachOccurrence            \
                         self other                            self     other   
0                         NaN   NaN                             NaN       NaN   
1                         NaN   NaN                         1000000  1000000    
2                         NaN   NaN                         1000000  1000000    
3                         NaN   NaN                         1000000  1000000    
4                         NaN   NaN                         1000000  1000000    
..                        ...   ...                             ...       ...   
147                       NaN   NaN                             NaN       NaN   
148                       NaN   NaN                             NaN       NaN   
149                       NaN   NaN                             NaN       NaN   
150                       NaN   NaN                         5000000   3000000   
151                       NaN   NaN                             NaN       NaN   

    GeneralLiability_GeneralAggregate           AutoLiability_Required        \
                                 self     other                   self other   
0                             2000000       NaN                    NaN   NaN   
1                                 NaN       NaN                    NaN   NaN   
2                                 NaN       NaN                    NaN   NaN   
3                             2000000  2000000                     NaN   NaN   
4                                 NaN       NaN                    NaN   NaN   
..                                ...       ...                    ...   ...   
147                               NaN       NaN                    NaN   NaN   
148                               NaN       NaN                    NaN   NaN   
149                               NaN       NaN                    NaN   NaN   
150                          10000000   6000000                    NaN   NaN   
151                               NaN       NaN                    Yes   NaN   

    AutoLiability_Conditional       WorkersCompensation_Required        \
                         self other                         self other   
0                         NaN   NaN                          NaN   NaN   
1                         NaN   NaN                          NaN   NaN   
2                         NaN   NaN                          NaN   NaN   
3                         Yes   NaN                          NaN   NaN   
4                         NaN   NaN                          NaN   NaN   
..                        ...   ...                          ...   ...   
147                       NaN   NaN                          NaN   Yes   
148                       NaN   NaN                          NaN   NaN   
149                       NaN   NaN                          NaN   NaN   
150                       NaN   NaN                          NaN   NaN   
151                       NaN   NaN                          NaN   NaN   

    WorkersCompensation_Conditional       ProfessionalLiability_Required  \
                               self other                           self   
0                               NaN   NaN                            NaN   
1                               NaN   NaN                            NaN   
2                               NaN   NaN                            Yes   
3                               NaN   NaN                            Yes   
4                               NaN   NaN                            Yes   
..                              ...   ...                            ...   
147                             NaN   NaN                            NaN   
148                             NaN   NaN                            NaN   
149                             NaN   NaN                            Yes   
150                             NaN   NaN                            Yes   
151                             Na

## Compare Results

In [21]:
total_elements = df.size
num_differences = comparison.count().sum() / 2 
num_matches = total_elements - num_differences
percentage_match = (num_matches / total_elements) * 100
print(f"Percentage of matching elements: {percentage_match}%")

Percentage of matching elements: 80.42763157894737%
